## Wk6 Assignment

## Lincoln Brown

## DSC550-T302

## Professor Werner

## Part 1: PCA and Variance Threshold in a Linear Regression

In [155]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split

## Step 1: Import the Data
Import the housing data as a dataframe and ensure that the data is loaded properly.

In [36]:
if not os.getcwd().__contains__('house-prices-advanced'):
    os.chdir('house-prices-advanced-regression-techniques')
os.listdir()

['test.csv', 'data_description.txt', 'train.csv', 'sample_submission.csv']

In [37]:
df = pd.read_csv('train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## Step 2: Drop Columns

Drop the "Id" column and any features that are missing more than 40% of their values.

In [144]:
cleaned_df = df.copy()
print(f"There are {cleaned_df.shape[0]} records and {cleaned_df.shape[1]} features in the dataset.")
if "Id" in cleaned_df.columns:
    cleaned_df = cleaned_df.drop("Id", axis=1)
cleaned_df.isna().sum()

There are 1460 records and 81 features in the dataset.


MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [145]:
# Drop any features that are missing more than 40% of their values
print("Dropping any features with more than 40% of their values missing.")
total_length = df.shape[0]
for col in cleaned_df.columns:
    col_nulls = cleaned_df.loc[:,col].isna().sum()
    percent_null = col_nulls / total_length
    if percent_null > .4:
        print(f"{col} # of NaNs: {col_nulls}.  {percent_null:.2%}")
        print(f"Dropping {col}")
        cleaned_df = cleaned_df.drop(col, axis=1)

Dropping any features with more than 40% of their values missing.
Alley # of NaNs: 1369.  93.77%
Dropping Alley
MasVnrType # of NaNs: 872.  59.73%
Dropping MasVnrType
FireplaceQu # of NaNs: 690.  47.26%
Dropping FireplaceQu
PoolQC # of NaNs: 1453.  99.52%
Dropping PoolQC
Fence # of NaNs: 1179.  80.75%
Dropping Fence
MiscFeature # of NaNs: 1406.  96.30%
Dropping MiscFeature


In [146]:
print(f"After cleaning, there are {cleaned_df.shape[0]} records and {cleaned_df.shape[1]} features.")

After cleaning, there are 1460 records and 74 features.


## Step 3: Fill in Missing Numeric Values

For numerical columns, fill in any missing data with the median value.

In [147]:
cleaned_df.dtypes.value_counts()

object     37
int64      34
float64     3
Name: count, dtype: int64

In [148]:
cleaned_df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [149]:
# To impute the NaNs in the numeric columns, we need to first select those columns
numeric_cols = cleaned_df.select_dtypes(include=['number']).columns
print(f"There are {len(numeric_cols)} numeric columns")

# Get the median for each numeric column
medians = cleaned_df.loc[:,numeric_cols].median() 
# Get a dataframe mask of true false values for NaNs. NaNs will be True, Non-missing values will be False
numeric_mask = cleaned_df.loc[:,numeric_cols].isna()

# pandas dataframe.where statements replace values marked as false, with corresponding value from other.
# Therefore, we need to bitwise NOT the mask, making the NaNs marked as True, False thus replacing them with medians
cleaned_df.loc[:,numeric_cols] = cleaned_df.loc[:,numeric_cols].where(cond=~numeric_mask, other=medians, axis=1)


There are 37 numeric columns


In [150]:
# Print the columns with NaNs
nans = cleaned_df.isna().sum()
nans[nans > 0]

BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

## Fill in Missing Categorical Values

For categorical columns, fill in any missing data with the most common value (mode).

In [151]:
# Now we will apply the same principle to the categorical columns
# First we isolate the categoric columns
categoric_cols = cleaned_df.select_dtypes(include=['object']).columns
print(f"There are {len(categoric_cols)} categorical columns")
# Get the mode for each categorical column
modes = cleaned_df.loc[:,categoric_cols].mode()
# Create a mask for the categoric columns marking NaNs as True and non-missing values as False
categoric_mask = cleaned_df.loc[:, categoric_cols].isna()

# Use a negated where statement to apply the modes to the categorical NaN values
cleaned_df.loc[:, categoric_cols] = cleaned_df.loc[:,categoric_cols].where(cond=~categoric_mask, other=modes, axis=1)

There are 37 categorical columns


In [152]:
# Confirm there are no more columns with NaNs
nans = cleaned_df.isna().sum()
nans[nans > 0]

Series([], dtype: int64)

## Convert to Dummy Variables

Convert the categorical columns to dummy variables.

In [153]:
dummy_df = pd.get_dummies(data=cleaned_df)

In [154]:
dummy_df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,False,False,False,True,False,False,False,False,True,False
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,False,False,False,True,False,False,False,False,True,False
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,False,False,False,True,False,False,False,False,True,False
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,False,False,False,True,True,False,False,False,False,False
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,False,False,False,True,False,False,False,False,True,False
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,False,False,False,True,False,False,False,False,True,False
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,False,False,False,True,False,False,False,False,True,False
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,False,False,False,True,False,False,False,False,True,False


## Step 6: Split the data

Split the data into a training and test set, where the SalePrice column is the target.

In [157]:
# Split features and target variable

# Get features
X = dummy_df.drop('SalePrice', axis=1) 

# Get Target Variable
y = dummy_df['SalePrice'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
